In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import os
import json
import re

# Path to the main dataset directory
dataset_path = '/content/drive/MyDrive/Policy_Bot_Data/dataset'

# Initialize list to store all chunks
all_chunks = []
folder_count = 0
pdf_count = 0

# Function to clean folder and file names
def clean_name(name):
    # Remove parentheses and their content
    name = re.sub(r'\s*\([^)]*\)', '', name)
    # Remove .pdf extension
    name = re.sub(r'\.pdf$', '', name, flags=re.IGNORECASE)
    return name.strip()

# Function to process PDF
def process_pdf(pdf_path, folder_name):
    global pdf_count
    try:
        # Loading pdf
        loader = PyPDFLoader(file_path=pdf_path)
        docs_before_split = loader.load()

        # Check if the document was loaded properly
        if len(docs_before_split) == 0:
            print(f"Warning: No content found in {pdf_path}")
            return

        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=700,
            chunk_overlap=50,
        )

        # Split the documents into chunks
        docs_after_split = text_splitter.split_documents(docs_before_split)

        # Check if the document was split correctly
        if len(docs_after_split) == 0:
            print(f"Warning: No chunks created from {pdf_path}")
            return

        # Clean the folder and file names
        clean_folder_name = clean_name(folder_name)
        clean_file_name = clean_name(os.path.basename(pdf_path))

        # Prepare the chunks in a dictionary format, including the cleaned folder and file name
        for i, doc in enumerate(docs_after_split):
            chunk_data = {
                'folder_name': clean_folder_name,
                'file_name': clean_file_name,
                'chunk_id': i + 1,
                'content': doc.page_content
            }
            all_chunks.append(chunk_data)
        pdf_count += 1
        print(f"Processed {pdf_path}, created {len(docs_after_split)} chunks.")
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")

# Walk through the main dataset directory and process all PDF files
for root, dirs, files in os.walk(dataset_path):
    if files:
        folder_count += 1  # Increment folder count when files are present
    for file_name in files:
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(root, file_name)
            folder_name = os.path.basename(root)  # Get the folder name for indexing
            process_pdf(pdf_path, folder_name)

# Check if there are any chunks to save
if all_chunks:
    # Save all chunks to a JSON file
    output_path = '/content/chunked_data_all_folders_cleaned.json'
    with open(output_path, 'w') as json_file:
        json.dump(all_chunks, json_file, indent=4)
    print(f"All PDF chunks have been saved to {output_path}")
else:
    print("No chunks were created. Please check the input files.")

# Print the number of folders traversed and the number of PDFs processed
print(f"Number of folders traversed: {folder_count}")
print(f"Number of PDFs processed: {pdf_count}")


Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Health & Wellness (HWD)/HWD-01 Wellness Policy .pdf, created 220 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Health & Wellness (HWD)/HWD-02 Phys Ed & Physical Activity.pdf, created 45 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Health & Wellness (HWD)/HWD-03 Comprehensive Health Ed.pdf, created 27 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Health & Wellness (HWD)/HWD-04 Healthy School Environment Policy.pdf, created 19 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Health & Wellness (HWD)/HWD-06 Tobacco-Nicotine Policy.pdf, created 35 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Enrollment Planning and Support (AMT)/AMT-01 Exam School Application and Admissions.pdf, created 38 chunks.
Processed /content/drive/MyDrive/Policy_Bot_Data/dataset/Enrollment Planning and Support (AMT)/AMT-03 DYS Committed Students.pdf, created 26 chunks